# Shark attacks data cleaning

- Importación de las librerías necesarias

In [1]:
import pandas as pd
import numpy as np
import re
import Src.SharkFn as Fn

- Creación del data frame Madre

In [2]:
df_Madre = pd.read_csv('Input/GSAF5.csv', encoding = "ISO-8859-1")

In [3]:
df_Madre.columns.values

array(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area',
       'Location', 'Activity', 'Name', 'Sex ', 'Age', 'Injury',
       'Fatal (Y/N)', 'Time', 'Species ', 'Investigator or Source', 'pdf',
       'href formula', 'href', 'Case Number.1', 'Case Number.2',
       'original order', 'Unnamed: 22', 'Unnamed: 23'], dtype=object)

In [4]:
print (df_Madre.shape)

(5992, 24)


In [5]:
null_cols = df_Madre.isnull().sum()
null_cols
null_cols[null_cols > 0]

Country                     43
Area                       402
Location                   496
Activity                   527
Name                       200
Sex                        567
Age                       2681
Injury                      27
Fatal (Y/N)                 19
Time                      3213
Species                   2934
Investigator or Source      15
href formula                 1
href                         3
Unnamed: 22               5991
Unnamed: 23               5990
dtype: int64

In [6]:
set(df_Madre['Activity']) # **!**
# set(df_Madre['Time']) # **!**
# set(df_Madre['Species ']) # **!**
# set(df_Madre['Country']) # **!**

# print(df_Madre['Type'].value_counts())
# print(df_Madre['Sex '].value_counts())
# print(df_Madre['Fatal (Y/N)'].value_counts())

# set(df_Madre['Date'])
# set(df_Madre['Year'])

# set(df_Madre['Area'])
# set(df_Madre['Location'])

# set(df_Madre['Name'])
# set(df_Madre['Age'])
# set(df_Madre['Injury'])

    #-----------#
#          set(df_Madre['Case Number'])
        
        # set(df_Madre['Investigator or Source'])
        # set(df_Madre['pdf'])
        # set(df_Madre['href formula'])
        # set(df_Madre['href'])
        # set(df_Madre['Case Number.1'])
        # set(df_Madre['Case Number.2'])
        # set(df_Madre['original order'])
        # set(df_Madre['Unnamed: 22'])
        # set(df_Madre['Unnamed: 23'])

{nan,
 'Swimming crawl stroke',
 'Diving for coins thrown from ship S.S. Moeraki',
 'Stuffing a shark into an automobile',
 "Lying prone in 2' of water",
 'Fishing (Seining)',
 'Crossing river on a raft',
 'Murder victim',
 'Washed overboard from the barque Mary Eady',
 'Removing fish from a trap',
 'Goggle-diving for seaweeds, but standing in water',
 'Free diving  & spearfishing (submerged)',
 'Dived for a coin',
 'Columbian petrol barge Rio Atrato burned and sank',
 'Fishing, holding fish',
 'Shell Diving',
 'Scuba diving & U/W photography',
 'Sinking of the dredge World Atlas',
 'Attempting to attract dolphins',
 'Swimming, attacked at  surf carnival',
 'Crew swimming alongside their anchored ship',
 'Jumped overboard after murdering 2 shipmates',
 'Fishingat',
 'Picking up shark by the tail',
 'Cutter capsized',
 'Helmet diving, collecting trochus shell',
 'Went overboard',
 'Speared a grouper, saw shark but it came for him instead of the fish so he fired spear into shark\x92s mou

In [7]:
to_drop = ['Investigator or Source','pdf','href formula','href',
           'Case Number.1','Case Number.2','original order','Unnamed: 22','Unnamed: 23']
df_raw = df_Madre
df_raw.drop(to_drop, inplace=True, axis=1)


In [8]:
df_raw.rename(columns = {'Species ': 'Species',
                        'Sex ': 'Sex',
                        'Fatal (Y/N)' : 'Fatal'}, inplace=True)
df_raw.columns.values

array(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area',
       'Location', 'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal',
       'Time', 'Species'], dtype=object)

In [9]:
def clean_last_ws (serie):
    return serie.replace("\s$", "")

def clean_first_ws (serie):
    return serie.replace("^\s", "")

def clean_mult_ws (serie):
    return serie.replace("\s+", " ")

def clean_weird_ch (serie):
    return serie.replace("[\[\]\,\"\'\\\.\-\?\<\>\*\&\;\+\(\)]", "")

def clean_num (serie):
    return serie.replace("[\d]", "")

def clean_1to2_ch (serie):
    aux = serie.replace("\s\w{1,2}\s", " ")
    aux1 = aux.replace("\s\w{1,2}\s", " ")
    aux2 = aux1.replace("\s\w{1,2}\s", " ")
    return aux2.replace("\s\w{1,2}\s", " ")
    
def shark_sp (serie):
    return serie.replace(r'(?!\w+\sshark)\b([\S\s]+?)(\b|$)', lambda x: (x.end() - x.start())*'')
    

Limpiando Sex

In [10]:
df_raw.Sex = Fn.clean_last_ws(df_raw.Sex.str)

df_raw.loc[(df_raw.Sex == '.')|
           (df_raw.Sex == 'N')|
           (df_raw.Sex == 'lli'), 'Sex'] = np.nan

df_raw.Sex.value_counts(dropna=False)


M      4837
F       585
NaN     570
Name: Sex, dtype: int64

Limpiando Fatal

In [11]:
df_raw.Fatal = Fn.clean_first_ws(df_raw.Fatal.str)
df_raw.Fatal = Fn.clean_last_ws(df_raw.Fatal.str)
df_raw.Fatal = df_raw.Fatal.str.capitalize()
df_raw.loc[(df_raw.Fatal == '#value!')|
           (df_raw.Fatal == 'F')|
           (df_raw.Fatal =='Unknown'), 'Fatal'] = np.nan

df_raw.Fatal.value_counts(dropna=False)

N      4325
Y      1552
NaN     115
Name: Fatal, dtype: int64

Limpiando Species

In [12]:
df_raw.Species = Fn.clean_weird_ch(df_raw.Species.str)
df_raw.Species = Fn.clean_1to2_ch(df_raw.Species.str)
df_raw.Species = Fn.clean_num(df_raw.Species.str)
df_raw.Species = Fn.clean_mult_ws(df_raw.Species.str)
df_raw.Species = Fn.clean_last_ws(df_raw.Species.str)
df_raw.Species = Fn.clean_first_ws(df_raw.Species.str)

# set(df_raw['Species'])

In [13]:
df_raw.loc[(df_raw.Species == ''), 'Species'] = np.nan

df_raw.Species.fillna('shark', inplace = True)
df_raw.Species = df_raw.Species.str.lower()

df_raw.Species = df_raw.Species.str.findall(r"\w+\sshark")

lst_cc_sp = []
for e in df_raw.Species:
    if e == []:
        lst_cc_sp.append("undef. shark")
    else:
        lst_cc_sp.append(e[0])

df_raw["Shark_sp"] = lst_cc_sp
# df_raw.Shark_sp.value_counts()

df_raw.loc[(df_raw.Shark_sp == 'to shark')|
           (df_raw.Shark_sp == 'small shark')|
           (df_raw.Shark_sp == 'm shark')|
           (df_raw.Shark_sp == 'involve shark')|
           (df_raw.Shark_sp == 'kg shark')|
           (df_raw.Shark_sp =='lb shark'), 'Shark_sp'] = "undef. shark"

# df_raw.Shark_sp.value_counts()

Limpiando Activity

In [14]:
df_raw.Activity.fillna('undefing', inplace = True)
df_raw.Activity = df_raw.Activity.str.lower()

df_raw.Activity = df_raw.Activity.str.findall(r"\w+ing")


df_raw.Activity

lst_cc_act = []
for e in df_raw.Activity:
    if e == []:
        lst_cc_act.append("undef. act")
    else:
        lst_cc_act.append(e[0])

df_raw["Deadly_act"] = lst_cc_act

df_raw.loc[(df_raw.Deadly_act == 'undefing'), 'Deadly_act'] = "undef. act"

df_raw.loc[(df_raw.Deadly_act == 'spearfishing')|
           (df_raw.Deadly_act == 'netting'), 'Deadly_act'] = "fishing"

df_raw.loc[(df_raw.Deadly_act == 'bathing')|
           (df_raw.Deadly_act == 'wading'), 'Deadly_act'] = "swimming"

df_raw.loc[(df_raw.Deadly_act == 'snorkeling')|
           (df_raw.Deadly_act == 'freediving'), 'Deadly_act'] = "diving"


df_raw.Deadly_act.value_counts()





swimming         1386
fishing          1093
surfing          1013
undef. act        894
diving            605
standing          128
boarding          127
floating           46
treading           39
kayaking           35
skiing             34
attempting         30
playing            25
walking            24
sitting            21
rowing             21
windsurfing        20
paddling           16
sinking            14
canoeing           13
sailing            13
collecting         12
skindiving         11
washing            11
splashing           9
during              9
feeding             8
boating             8
clinging            8
removing            7
                 ... 
dynamiting          1
inspecting          1
murdering           1
lobstering          1
bending             1
checking            1
sleeping            1
painting            1
longling            1
expecting           1
seeing              1
missing             1
ascending           1
exercising          1
fleeing   

In [15]:
# df_raw.Sex = df_raw.Sex.str.replace("\s$", "")
# print(df_Madre['Sex '].value_counts())

In [16]:
text = 'bla bla diving sharking blabla'

textCC = re.findall(r'\w+ing', text)
textCC

['diving', 'sharking']

In [17]:
# df_raw = df_Madre

# def fun(*args,**kwargs):
#     print("Generic FN:")
#     print("\tArgs", args)
#     print("\tKwargs", kwargs)
    
# def clean_last_ws (serie):
#     aux = re.sub("\s$", "", serie)
#     return aux
    
# new = df_raw[["Sex "]].values.tolist()
# new
# aux = re.sub("\s$", "", *new)
# aux